# Evaluate classifier and key recovery
This notebook evaluates the performance of classifier and key search algorithm for messages encrypted under the caesar cipher.

In [ ]:
import sys
sys.path.append('src')

from src.eval_utils import *
from src.data_utils import *
import src.alphabet as alph
import src.cipher as Cipher
import src.cracker as Cracker
import seaborn as sns
import random
from sklearn.metrics import balanced_accuracy_score

In [ ]:
# Set parameters
data_id = 'alice-test'
alphabet = alph.basic_lower()
alphabet_len = len(alphabet)
cipher_proportions = [1, 5, 10]
cipher_proportions = [1]
ciphers = {'plain': 1, 'caesar': 1}
max_length = 50

## Evaluate model over complete message space

In [ ]:
# Evaluate model accuracy plain vs cipher
dl_test, _ = gen_dl(data_id, alphabet, 10, 100, ciphers=ciphers, key_rot=1, lower=True)
trainer = Trainer(nb_epochs=1)
model_path = f'models/alice_caesar_prop1.cnn'
model = load_model(model_path, alphabet_len)
score = trainer.score(model, dl_test, balanced_accuracy_score)
print(f'Model has balanced accuracy of {score}')

In [ ]:
# Evaluate accuracy of key recovery

# Generate samples
n_samples = 500
msgs = load_msgs(data_id, alphabet, 10, 100, lower=True)
order = get_char_order(msgs, alphabet)
samples = random.sample(msgs, n_samples)

# Generate caesar cracker
cipher = Cipher.Caesar(alphabet)
cracker = Cracker.Caesar(alphabet, model)

# Evaluate cracker
acc = evaluate_cracker(model_path, samples, order, cipher, cracker, lower=True, balanced=True)
print(f'Model has recovered {acc*100}% of keys')

## Evaluate model over different message lengths

In [ ]:
# Evaluate accuracy of key recovery
accuracies = []
xlabels = []
for p in cipher_proportions:
    model_path = f'models/alice_caesar_prop{p}.cnn'
    label = f'prop{p}'
    acc = evaluate_cracker_across_bins(model_path, data_id, alphabet, max_length=max_length, lower=True, balanced=True)
    accuracies.append(acc)
    xlabels.append(label)

In [ ]:
# Plot results
compare_accuracies(accuracies, xlabels, 'Accuracy of key recovery on bible data', max_length=max_length, notes='')